![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/8.Generic_Classifier.ipynb)

# 8. Generic Classifier

In [ ]:
import json
import os
from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.4.1 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [ ]:
import json
import os

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 6.1.3
Spark NLP_JSL Version : 6.1.1


In [ ]:
 # if you want to start the session with custom params as in start function above

def start(secret):
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:"+version) \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+secret+"/spark-nlp-jsl-"+jsl_version+".jar")

    return builder.getOrCreate()


#spark = start(secret)

## Healthcare NLP for Data Scientists Course

If you are not familiar with the components in this notebook, you can check [Healthcare NLP for Data Scientists Udemy Course](https://www.udemy.com/course/healthcare-nlp-for-data-scientists/) and the [MOOC Notebooks](https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/Spark_NLP_Udemy_MOOC/Healthcare_NLP) for each components.

## load dataset

In [ ]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/petfinder-mini.csv

In [ ]:
import pandas as pd
dataframe = pd.read_csv('petfinder-mini.csv')

In [ ]:
# In the original dataset "4" indicates the pet was not adopted.
import numpy as np
dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)

In [ ]:
dataframe = dataframe.drop(['AdoptionSpeed'], axis=1)

In [ ]:
dataframe.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,target
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,1
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,1
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,1
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,1
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,1


In [ ]:
dataframe.columns

Index(['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
       'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'Description',
       'PhotoAmt', 'target'],
      dtype='object')

In [ ]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11537 entries, 0 to 11536
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Type          11537 non-null  object
 1   Age           11537 non-null  int64 
 2   Breed1        11537 non-null  object
 3   Gender        11537 non-null  object
 4   Color1        11537 non-null  object
 5   Color2        11537 non-null  object
 6   MaturitySize  11537 non-null  object
 7   FurLength     11537 non-null  object
 8   Vaccinated    11537 non-null  object
 9   Sterilized    11537 non-null  object
 10  Health        11537 non-null  object
 11  Fee           11537 non-null  int64 
 12  Description   11528 non-null  object
 13  PhotoAmt      11537 non-null  int64 
 14  target        11537 non-null  int64 
dtypes: int64(4), object(11)
memory usage: 1.3+ MB


In [ ]:
dataframe.target.value_counts()

,count
target,
1,8457
0,3080


In [ ]:
dataframe.Description = dataframe.Description.fillna('- no description -')

## Featurize with Sklearn Column Transformer

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

column_trans = make_column_transformer(
     (OneHotEncoder(), ['Type', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
       'FurLength', 'Vaccinated', 'Sterilized', 'Health']),
     (TfidfVectorizer(max_features=100,  norm='l2', ngram_range=(1, 3)), 'Description'),
     remainder=StandardScaler())

X = column_trans.fit_transform(dataframe.drop(['target'], axis=1))

y = dataframe.target

In [ ]:
y.nunique()

2

In [ ]:
X.shape

(11537, 302)

In [ ]:
input_dim = X.shape[1]

In [ ]:
input_dim

302

In [ ]:
df = pd.DataFrame.sparse.from_spmatrix(X)

df.columns = ['col_{}'.format(i) for i in range(input_dim)]

df['target']= y

df.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_293,col_294,col_295,col_296,col_297,col_298,col_299,col_300,col_301,target
0,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-0.452479,0.950288,-0.829762,1
1,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-0.555981,-0.299388,-0.511871,1
2,0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-0.555981,-0.299388,1.077583,1
3,0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.22648,0,-0.400729,1.575125,1.395473,1
4,0,1.0,0,0,0,0,0,0,0,0,...,0,0.198113,0,0.133816,0.181835,0,-0.555981,-0.299388,-0.19398,1


## Train with Spark NLP Generic Classifier

**Building a pipeline**

The FeaturesAssembler is used to collect features from different columns. It can collect features from single value columns (anything which can be cast to a float, if casts fails then the value is set to 0), array columns or SparkNLP annotations (if the annotation is an embedding, it takes the embedding, otherwise tries to cast the 'result' field). The output of the transformer is a FEATURE_VECTOR annotation (the numeric vector is in the 'embeddings' field).

The GenericClassifierApproach takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations. The operation of the classifier is controled by the following methods:

*setEpochsNumber(int)* - Determines how many epochs the model is trained.

*setBatchSize(int)* - Sets the batch size during training.

*setLearningRate(float)* - Sets the learning rate.

*setValidationSplit(float)* - Sets the proportion of examples in the training set used for validation.

*setModelFile(string)* - Loads a model from the specified location and uses it instead of the default model.

*setFixImbalance(boolean)* - If set to true, it tries to balance the training set by weighting the classes according to the inverse of the examples they have.

*setFeatureScaling(string)* - Normalizes the feature factors using the specified method ("zscore", "minmax" or empty for no normalization).

*setOutputLogsPath(string)* - Sets the path to a folder where logs of training progress will be saved. No logs are generated if no path is specified.

In [ ]:
spark_df = spark.createDataFrame(df)
spark_df.select(spark_df.columns[-10:]).show(2)

+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
|col_293|col_294|col_295|col_296|col_297|col_298|            col_299|             col_300|            col_301|target|
+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0|-0.4524794726808656|  0.9502875792756131|-0.8297616989552165|     1|
|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0| -0.555981017719065|-0.29938816657135553|-0.5118709929431844|     1|
+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
only showing top 2 rows



In [ ]:
from pyspark.sql.functions import col

spark_df.groupBy("target") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+------+-----+
|target|count|
+------+-----+
|     1| 8457|
|     0| 3080|
+------+-----+



In [ ]:
(training_data, test_data) = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 9185
Test Dataset Count: 2352


## Create a custom DL architecture with TF

In [ ]:
!pip install git+https://github.com/tensorflow/addons.git

In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

In [ ]:
graph_folder = "gc_graph"

gc_graph_builder = TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("target")\
    .setHiddenLayers([300, 200, 100])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph.pb")

In [ ]:
'''
# Create custom graph

# If this method is used, graph folder should be added to training
# model as `.setGraphFolder(graph_folder_path)`

from sparknlp_jsl.training import tf_graph

!mkdir gc_graph

tf_graph.print_model_params("generic_classifier")

DL_params = {"input_dim": input_dim,
             "output_dim": y.nunique(),
             "hidden_layers": [300, 200, 100],
             "hidden_act": "tanh",
             'hidden_act_l2':1,
             'batch_norm':1}


tf_graph.build("generic_classifier",
               build_params=DL_params,
               model_location="/content/gc_graph",
               model_filename="auto")
'''

'''
# or just use the one we already have in the repo

!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/generic_classifier_graph/pet.in1202D.out2.pb -P /content/gc_graph
'''

In [ ]:
from sparknlp_jsl.base import *

!mkdir logs

features_asm = FeaturesAssembler()\
    .setInputCols(['col_{}'.format(i) for i in range(X.shape[1])])\
    .setOutputCol("features")

gen_clf = GenericClassifierApproach()\
    .setLabelColumn("target")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph.pb")\
    .setEpochsNumber(50)\
    .setBatchSize(100)\
    .setFeatureScaling("zscore")\
    .setFixImbalance(True)\
    .setLearningRate(0.002)\
    .setOutputLogsPath("logs")\
    .setValidationSplit(0.2) # keep 20% of the data for validation purposes

clf_Pipeline = Pipeline(
    stages=[
        features_asm,
        gc_graph_builder,
        gen_clf
])

mkdir: cannot create directory ‘logs’: File exists


In [ ]:
%%time

# train 50 epochs (takes around 1 min)

clf_model = clf_Pipeline.fit(training_data)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph.pb
Build params: {'input_dim': 302, 'output_dim': 2, 'hidden_layers': [300, 200, 100], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}


/usr/local/lib/python3.12/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.19.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're u

generic_classifier graph exported to gc_graph/gcf_graph.pb
CPU times: user 710 ms, sys: 45.3 ms, total: 756 ms
Wall time: 25.7 s


In [ ]:
log_file_name = os.listdir("logs")[0]

with open("logs/"+log_file_name, "r") as log_file :
    print(log_file.read())

Training 50 epochs
Epoch 1/50	0.51s	Loss: 19.267607	ACC: 0.6256081	Validation ACC: 0.6483397
Epoch 2/50	0.27s	Loss: 16.23476	ACC: 0.7111486	Validation ACC: 0.66467065
Epoch 3/50	0.27s	Loss: 14.726174	ACC: 0.74364865	Validation ACC: 0.6581383
Epoch 4/50	0.25s	Loss: 12.876577	ACC: 0.79070944	Validation ACC: 0.6831791
Epoch 5/50	0.25s	Loss: 11.104715	ACC: 0.82611483	Validation ACC: 0.7093087
Epoch 6/50	0.25s	Loss: 9.05053	ACC: 0.86920035	Validation ACC: 0.7229178
Epoch 7/50	0.26s	Loss: 7.0223603	ACC: 0.9031641	Validation ACC: 0.6962439
Epoch 8/50	0.25s	Loss: 5.2261443	ACC: 0.93063045	Validation ACC: 0.72182906
Epoch 9/50	0.24s	Loss: 3.7890537	ACC: 0.9575338	Validation ACC: 0.7131192
Epoch 10/50	0.24s	Loss: 2.9478858	ACC: 0.9668581	Validation ACC: 0.7441481
Epoch 11/50	0.24s	Loss: 2.0371375	ACC: 0.9809347	Validation ACC: 0.7419706
Epoch 12/50	0.24s	Loss: 1.4439961	ACC: 0.9859348	Validation ACC: 0.73162764
Epoch 13/50	0.23s	Loss: 1.1546333	ACC: 0.98985386	Validation ACC: 0.7212847
Epoch 14/

In [ ]:
pred_df = clf_model.transform(test_data)

In [ ]:
pred_df.select('target','prediction.result').show()

+------+------+
|target|result|
+------+------+
|     1|   [0]|
|     1|   [0]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     0|   [0]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [0]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     0|   [1]|
|     1|   [0]|
+------+------+
only showing top 20 rows



In [ ]:
preds_df = pred_df.select('target','prediction.result').toPandas()

# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : int(x[0]))


In [ ]:
preds_df.sample(10)

,target,result
1702,0,1
1242,1,1
1211,1,0
467,0,1
593,1,1
1079,0,1
839,0,0
1915,1,1
536,1,0
1129,1,1


In [ ]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report, accuracy_score

print (classification_report(preds_df['target'], preds_df['result'], digits=4))

print (accuracy_score(preds_df['target'], preds_df['result']))


              precision    recall  f1-score   support

           0     0.4547    0.4926    0.4729       611
           1     0.8166    0.7926    0.8044      1741

    accuracy                         0.7147      2352
   macro avg     0.6356    0.6426    0.6387      2352
weighted avg     0.7226    0.7147    0.7183      2352

0.7147108843537415


## get prediction for random input

In [ ]:
pd.DataFrame([dataframe.loc[5191].to_dict()])

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,target
0,Dog,2,Mixed Breed,Female,Black,Brown,Medium,Medium,Yes,Yes,Healthy,10,Friendly and pampered female puppy looking for...,2,1


In [ ]:
input_X = column_trans.transform(pd.DataFrame([dataframe.loc[0].to_dict()]).drop(['target'], axis=1))

input_y = dataframe.target[0]


In [ ]:
input_df = pd.DataFrame.sparse.from_spmatrix(input_X)

input_df.columns = ['col_{}'.format(i) for i in range(input_X.shape[1])]

input_df['target']= input_y

input_df.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_293,col_294,col_295,col_296,col_297,col_298,col_299,col_300,col_301,target
0,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-0.452479,0.950288,-0.829762,1


In [ ]:
input_spark_df = spark.createDataFrame(input_df)
input_spark_df.show(2)

+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-----

In [ ]:
clf_model.transform(input_spark_df).select('target','prediction.result').show()

+------+------+
|target|result|
+------+------+
|     1|   [1]|
+------+------+



# Case Study: Alexa Review Classification

In [ ]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/amazon_alexa.tsv

In [ ]:
import pandas as pd
df = pd.read_csv('amazon_alexa.tsv', sep='\t')
df

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1
...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in betwe...",1
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, check...",1
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my...",1
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1


In [ ]:
df.verified_reviews = df.verified_reviews.str.lower()

In [ ]:
df.feedback.value_counts()

,count
feedback,
1,2893
0,257


In [ ]:
df = df.copy()
df['verified_reviews'] = df['verified_reviews'].fillna('').astype(str).str.strip()

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


column_trans = make_column_transformer((OneHotEncoder(), ['rating','variation']),
     (TfidfVectorizer(max_features=1000,  norm='l2', ngram_range=(1, 3)), 'verified_reviews'))

X = column_trans.fit_transform(df.drop(['feedback'], axis=1))

y = df.feedback

In [ ]:
sdf = pd.DataFrame.sparse.from_spmatrix(X)

sdf.columns = ['col_{}'.format(i) for i in range(X.shape[1])]

sdf['feedback']= y

sdf.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_1012,col_1013,col_1014,col_1015,col_1016,col_1017,col_1018,col_1019,col_1020,feedback
0,0,0,0,0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,1.0,0,0,0,0,0,0,...,0,0.324472,0,0.149662,0,0,0,0,0,1
3,0,0,0,0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
input_spark_df = spark.createDataFrame(sdf)

In [ ]:
input_spark_df.show(5)

+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------------------+-------------------+-------------------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------------------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------------------+------+------+------+------+------+------+------+------+------+-------------------+------+------+------+------+------+------+------+------+------+------+-------------------+-------------------+------+------+------+------+-------+-------+-------+-------------------+-------+-------+-------+-------+-------+-------+-------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-----

In [ ]:
(training_data, test_data) = input_spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 2478
Test Dataset Count: 672


In [ ]:

from pyspark.ml import Pipeline
from sparknlp_jsl.base import *

input_cols = [c for c in training_data.columns if c.startswith('col_')]

features_asm = FeaturesAssembler() \
    .setInputCols(input_cols) \
    .setOutputCol("features")

gc_graph_builder = TFGraphBuilder() \
    .setModelName("generic_classifier") \
    .setInputCols(["features"]) \
    .setLabelColumn("feedback") \
    .setHiddenLayers([300, 200, 100]) \
    .setHiddenAct("tanh") \
    .setHiddenActL2(True) \
    .setBatchNorm(True) \
    .setGraphFolder(graph_folder) \
    .setGraphFile("gcf_graph.pb")

gen_clf = GenericClassifierApproach() \
    .setLabelColumn("feedback") \
    .setInputCols(["features"]) \
    .setOutputCol("prediction") \
    .setModelFile(f"{graph_folder}/gcf_graph.pb") \
    .setEpochsNumber(50) \
    .setBatchSize(100) \
    .setFeatureScaling("zscore") \
    .setFixImbalance(True) \
    .setLearningRate(0.001) \
    .setOutputLogsPath("logs")

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf
])

clf_model = clf_Pipeline.fit(training_data)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph.pb
Build params: {'input_dim': 1021, 'output_dim': 2, 'hidden_layers': [300, 200, 100], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph.pb


In [ ]:
pred_df = clf_model.transform(test_data)

preds_df = pred_df.select('feedback','prediction.result').toPandas()

# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : int(x[0]))

# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(preds_df['feedback'], preds_df['result'], digits=4))

print(accuracy_score(preds_df['feedback'], preds_df['result']))


              precision    recall  f1-score   support

           0     0.5714    0.9796    0.7218        49
           1     0.9983    0.9422    0.9694       623

    accuracy                         0.9449       672
   macro avg     0.7849    0.9609    0.8456       672
weighted avg     0.9672    0.9449    0.9514       672

0.9449404761904762


## Extra Informations

**ExceptionHandling**

A robust exception-handling mechanism is available to ensure the process continues even if corrupted inputs are encountered. When set to True, the annotator attempts to process the data as usual. If exception-causing data (e.g., a corrupted record or document) is passed, a warning is emitted with the exception message. Processing then continues with the next record, while the rest of the batch is parsed without interruption. Note that this option may incur a performance penalty. By default, this behavior is False, meaning the process will throw an exception and terminate to inform users of the issue.


*Example*:
```python
gen_clf = GenericClassifierApproach()\
    .setLabelColumn("feedback")\
    .setInputCols(["features"])\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph.pb")\
    .setEpochsNumber(50)\
    .setBatchSize(100)\
    .setFeatureScaling("zscore")\
    .setFixImbalance(True)\
    .setLearningRate(0.001)\
    .setOutputLogsPath("logs")\
    .setDoExceptionHandling(True)
    
```